# NYC Ticket Writing Machine!
### DS4400 Final Project
### by Benjamin Kosiborod and Victoria Staada
&nbsp;
## Data Cleanup

In [70]:
# All project imports
import pandas as pd
import numpy as np
import math
import datetime
import re

In [3]:
# Read in the data
data = pd.read_csv('https://data.cityofnewyork.us/resource/faiq-9dfq.csv?$limit=12000000')

/home/ood.discovery.neu.edu/software/anaconda3/2019.10/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (17,18,20,22,23,29,30,31,32,36,38,39) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
# Copy the data so that we can
# manipulate the data and re-copy
# if we make a mistake
df = data.copy()

In [42]:
# Preview the data
print(df.describe())

       summons_number  violation_code  street_code1  street_code2  \
count    1.146751e+07    1.146751e+07  1.146751e+07  1.146751e+07   
mean     7.474217e+09    3.377908e+01  2.562781e+04  2.119624e+04   
std      2.268085e+09    1.985783e+01  2.246802e+04  2.193509e+04   
min      1.028884e+09    0.000000e+00  0.000000e+00  0.000000e+00   
25%      8.500714e+09    2.000000e+01  9.130000e+03  0.000000e+00   
50%      8.655880e+09    3.600000e+01  1.934000e+04  1.474000e+04   
75%      8.694870e+09    4.000000e+01  3.618000e+04  3.427000e+04   
max      8.768851e+09    9.900000e+01  9.802000e+04  9.831000e+04   

       street_code3  vehicle_expiration_date  violation_location  \
count  1.146751e+07             1.146751e+07        9.589591e+06   
mean   2.126681e+04             2.691894e+07        5.636025e+01   
std    2.198500e+04             2.736702e+07        3.963513e+01   
min    0.000000e+00             0.000000e+00        1.000000e+00   
25%    0.000000e+00             2.0181

In [43]:
# Drop some columns that are mostly empty, 
# or do not have useful data
df = df.drop(columns=['violation_post_code', 'violation_location', 'feet_from_curb', 'house_number', 'intersecting_street', 'date_first_observed', 'law_section', 'sub_division', 'violation_legal_code', 'violation_in_front_of_or', 'time_first_observed', 'issuer_code', 'issuer_command', 'issuer_squad', 'summons_number', 'plate_id', 'days_parking_in_effect', 'from_hours_in_effect', 'to_hours_in_effect', 'unregistered_vehicle', 'violation_description', 'no_standing_or_stopping', 'hydrant_violation', 'double_parking_violation'])

In [44]:
# Check columns left and their types
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11467506 entries, 0 to 11467505
Data columns (total 20 columns):
registration_state         object
plate_type                 object
issue_date                 object
violation_code             int64
vehicle_body_type          object
vehicle_make               object
issuing_agency             object
street_code1               int64
street_code2               int64
street_code3               int64
vehicle_expiration_date    int64
violation_precinct         int64
issuer_precinct            int64
violation_time             object
violation_county           object
street_name                object
vehicle_color              object
vehicle_year               int64
meter_number               object
violation_post_code        object
dtypes: int64(8), object(12)
memory usage: 1.7+ GB
None


In [45]:
# Instead of having meter numbers, change this column into a boolean value
# for meter or no meter recorded at time of violation
df['meter_number'] = df['meter_number'].apply(lambda x: 0 if x == '-' or pd.isnull(x) else 1)
df = df.rename({'meter_number': 'meter?'}, axis=1)

In [46]:
colors = {}
for color in ('WH', 'W', 'w', 'white', 'White', 'WT', 'WHI', 'WH/', 'WHITE', 'Cream', 'CREAM', 'WT.', 'WHTE', 'WH YW', 'WHITW', 'HT', 'WHT', 'WHBL', 'WHB', 'WHO', 'WHGY', 'WHIT', 'WHG', 'WHRD', 'WHGR', 'WTE', 'WH.', 'WHBK', 'WHTN', 'WHT.', 'WHTIE', 'WHE', 'WHGL', 'W/B', 'CRM', 'WHBR', 'WHWH', 'WHOR', 'WG', 'WHYW', 'WHIE', 'WJ', 'WHLE'):
    colors[color] = 'White'
for color in ('Black', 'BLK', 'BK', 'black', 'BLK.', 'BK.', 'BLACK', 'BLW', 'BKGY', 'BK/', 'BKBL', 'BLCK', 'BKGR', 'BLWH', 'BLA', 'BKG', 'BLK.', 'BKTN', 'BKW', 'BKT', 'BKWH', 'BLAC', 'BLAK', 'BLTN', 'BLRD', 'BKBK', 'BLGL', 'BLKWH'):
    colors[color] = 'Black'
for color in ('GRY', 'Gray', 'GY/', 'GRAY', 'GY', 'GREY', 'grey', 'Grey', 'DKGRY', 'M.GRE', 'GY/GL', 'GRAYF', 'CHRAY', 'LTGY', 'DKGY', 'GYGY', 'GYBL', 'GYGR', 'GY.', 'GYB', 'GRA', 'GYRD', 'GYBK', 'GYTN', 'GYG', 'GYBR', 'GYWH', 'GRY.', 'GRAY.', 'ALUMI', 'GYT', 'GREY.', 'GYGL', 'Gray', 'GYPR', 'GY GR'):
    colors[color] = 'Gray'
for color in ('Silver', 'SILVER', 'SLV', 'SV', 'SL', 'SILV', 'SILVE', 'SIL', 'SILVR', 'SL.', 'STEEL', 'MET', 'SLVR', 'SLR', 'SIV', 'SLVER', 'SLIVE', 'SLVE', 'SIL.'):
    colors[color] = 'Silver'
for color in ('TAN', 'Beige', 'BEIGE', 'beige', 'BLD', 'ALMON', 'TN', 'LTTN', 'TNGY', 'BE', 'BIEGE', 'TNGR', 'DKTN', 'TN/', 'BEIG'):
    colors[color] = 'Beige'
for color in ('RED', 'red', 'RD', 'rd', 'Rd', 'Red', 'RO', 'BUGA', 'MAROO', 'MAR', 'MR', 'BUNGE', 'RDW', 'DKR', 'DKRD', 'RD/', 'BURG', 'BURGU', 'BUR', 'RDGY', 'RD.', 'RDT', 'RDBK', 'RDBL', 'MRPK', 'RDG', 'RDGR', 'RDWH', 'RDRD', 'RDTN', 'DKMR', 'RD BK', 'RED.', 'BURGA', 'MRGY', 'MRN', 'BUG', 'RE', 'RDBR', 'DKRED'):
    colors[color] = 'Red'
for color in ('BLUE', 'BL', 'BLLU', 'QBLUE', 'BUO', 'BLU', 'DKBL', 'BLG', 'BL/', 'BLGY', 'LTBL', 'BLGR', 'BLBL', 'DBL', 'BL.', 'BLB', 'LBL', 'BLBK', 'BLUE.', 'NAVY', 'BLRD'):
    colors[color] = 'Blue'
for color in ('GREEN', 'GR', 'GYN', 'GRN', 'LTGR', 'DKGR', 'GR/', 'GRE', 'GRGY', 'GRG', 'GRW', 'GRBL', 'GRGR', 'DGR', 'GRB', 'GREN', 'GRT', 'LGR', 'GREE', 'GRTN'):
    colors[color] = 'Green'
for color in ('YELLW', 'YEL', 'YELL', 'YELLO', 'YW', 'GOLDE', 'ORO', 'YOL', 'YLOW', 'YL', 'GL', 'GOLD', 'GLD', 'YLW', 'Y', 'YE', 'YLLW', 'YELLL'):
    colors[color] = 'Yellow'
for color in ('BROWN', 'BR', 'BR/GY', 'BON', 'BRWMN', 'BREIR', 'BEUG', 'BRWN', 'BRN', 'BRO', 'BRW', 'BWN', 'LTBR', 'BROW', 'BRZ', 'DKBR', 'BRBL', 'BRON', 'BRT'):
    colors[color] = 'Brown'
for color in ('LAVEN', 'PURPL', 'PR', 'PURP', 'PUR', 'DKRR'):
    colors[color] = 'Purple'
for color in ('OR', 'ORANGE', 'ORANG', 'ONG', 'O', 'OG', 'ORA', 'ORAN', 'ORWH', 'ORN'):
    colors[color] = 'Orange'

df['vehicle_color'] = df['vehicle_color'].apply(lambda row: colors.get(row, row))

# Drop rows that are not categorized by the above, 
# as there as <500K such records out of 11.5M
# and the colors begin to get more niche. We
# suspect there is not enough data to accurately
# make predictions for the remaining colors.
df = df[df.vehicle_color.isin(colors.values())]

In [47]:
df = df[(df.vehicle_year <= 2020) & (df.vehicle_year != 0)]

In [48]:
# Get rid of null and non-stated vehicle makes, 
# as well as rows that contain makes where 
# there are less than 100 records
df = df[pd.notnull(df.vehicle_make) 
        & (df.vehicle_make != 'NS/OT') 
        & (df.vehicle_make != 'FRE') 
        & (df.vehicle_make != 'STARC')
        & (df.vehicle_make != 'BL/BI')
        & (df.vehicle_make != 'COLLI')
        & (df.vehicle_make != 'UNIFL')
        & (df.vehicle_make != 'ORION')
        & (df.vehicle_make != 'TSM')
        & (df.vehicle_make != 'EASTO')]
df = df.groupby('vehicle_make').filter(lambda x: len(x) > 100)

makes = {
    'FORD': 'Ford',
    'TOYOT': 'Toyota',
    'HONDA': 'Honda',
    'NISSA': 'Nissan',
    'CHEVR': 'Chevrolet',
    'ME/BE': 'Mercedes-Benz',
    'FREUH': 'Fruehauf',
    'DODGE': 'Dodge',
    'HYUND': 'Hyundai',
    'JEEP': 'Jeep',
    'LEXUS': 'Lexus',
    'INTER': 'International Harvester',
    'ACURA': 'Acura',
    'INFIN': 'Infiniti',
    'SUBAR': 'Subaru',
    'VOLKS': 'Volkswagen',
    'HIN': 'Hindustan Motors',
    'CHRYS': 'Chrystler',
    'AUDI': 'Audi',
    'ISUZU': 'Isuzu',
    'KIA': 'Kia',
    'MAZDA': 'Mazda',
    'MITSU': 'Mitsubishi',
    'ROVER': 'Land Rover',
    'CADIL': 'Cadillac',
    'LINCO': 'Lincoln',
    'VOLVO': 'Volvo',
    'WORKH': 'Workhorse',
    'KENWO': 'Kenworth',
    'BUICK': 'Buick',
    'SMART': 'smart',
    'PETER': 'Peterbilt',
    'MINI': 'Mini',
    'PORSC': 'Porsche',
    'MERCU': 'Mercury',
    'JAGUA': 'Jaguar',
    'FIAT': 'Fiat',
    'SATUR': 'Saturn',
    'PONTI': 'Pontiac',
    'UTILI': 'Utility',
    'MACK': 'Mack',
    'TESLA': 'Tesla',
    'SUZUK': 'Suzuki',
    'UD': 'UD Trucks',
    'MASE': 'Maserati',
    'SAAB': 'Saab',
    'HYUN': 'Hyundai',
    'HUMME': 'Hummer',
    'OLDSM': 'Oldsmobile',
    'MCI': 'Motor Coach Industries',
    'THOMA': 'Thomas Built Busses',
    'IC': 'IC Bus',
    'YAMAH': 'Yamaha',
    'BENTL': 'Bentley',
    'VANHO': 'Van Hool Bus',
    'SPRI': 'Springdale RVs',
    'KAWAS': 'Kawasaki',
    'ALFAR': 'Alfa Romeo',
    'PLYMO': 'Plymouth',
    'HARLE': 'Harley Davidson',
    'STERL': 'Sterling',
    'VESPA': 'Vespa',
    'UPS': 'UPS Truck',
    'VPG': 'Vehicle Production Group',
    'PREVO': 'Prevost RVs',
    'GEO': 'Geo',
    'NAVIS': 'Navistar',
    'FERRA': 'Ferrari',
    'TRIUM': 'Triumph',
    'DUCAT': 'Ducati',
    'ZENIT': 'Zenith',
    'GENES': 'Genesis',
    'SCION': 'Scion',
    'HINO': 'Hino Motors',
    'ROLLS': 'Rolls Royce',
    'LAMBO': 'Lamborgini',
    'CHECK': 'Checker Motors',
    'SETRA': 'Setra Coach',
    'ACUR': 'Acura',
    'GREAT': 'Great Dane',
    'LEXU': 'Lexus',
    'SMITH': 'Smith Electric Vehicles',
}

df['vehicle_make'] = df['vehicle_make'].apply(lambda row: makes.get(row, row))

In [49]:
# clean up violation county
counties = {
    'NY': 'Manhattan',
    'K': 'Brooklyn',
    'Q': 'Queens',
    'BX': 'Bronx',
    'BK': 'Brooklyn',
    'QN': 'Queens',
    'R': 'Staten Island',
    'ST': 'Staten Island',
    'MN': 'Manhattan',
    'QUEEN': 'Queens',
}
df['violation_county'] = df['violation_county'].apply(lambda row: counties.get(row, row))

In [50]:
# clean up registration state
df = df[pd.notnull(df.registration_state) & (df.registration_state != '99')]
state = {
    'GV': 'U.S. Government',
    'DP': 'U.S. Department of State',
}
df['registration_state'] = df['registration_state'].apply(lambda row: state.get(row, row))

In [51]:
df = df[pd.notnull(df.plate_type) & (df.plate_type != '999')]
df = df.groupby('plate_type').filter(lambda x: len(x) > 100)
plate_types = {
    'PAS': 'Passenger',
    'COM': 'Commercial',
    'OMT': 'Taxi Omnibus',
    'OMS': 'Special Omnibus Rentals',
    'SRF': 'Special Passenger',
    'APP': 'Apportioned',
    'TRC': 'Tractor',
    'ORG': 'Organizational',
    'OMR': 'Omnibus',
    'MED': 'Medical Doctor',
    'SPO': 'Sports Passenger',
    'OML': 'Livery Omnibus',
    'PSD': 'Political Subdivision',
    'SCL': 'School Car',
    'IRP': 'International Reg Plan',
    'TOW': 'Tow Truck',
    'MOT': 'Motorcycle',
    'RGL': 'Regional',
    'VAS': 'Volunteer Ambulance Services',
    'ITP': 'In Transit Permit',
    'SRN': 'Special Passenger Judges/Officials',
    'HIS': 'Historical',
    'TRA': 'Transporter',
    'CHC': 'Household Carrier Commercial',
    'STA': 'State Agencies',
    'AGR': 'Agricultural',
    'AMB': 'Ambulance',
    'PHS': 'Pearl Harbor Survivors',
    'AYG': 'Army National Guard',
    'RGC': 'Regional Commercial',
    'ORC': 'Organizational Commercial',
    'TRL': 'Regular Trailer',
    'HAM': 'HAM Operator',
    'DLR': 'Dealer',
}
df['plate_type'] = df['plate_type'].apply(lambda row: plate_types.get(row, row))

In [52]:
# Clean up issue date, get rid of dates not in 2019 fiscal year
# (includes 2018 and 2019 issue dates)
df.issue_date = pd.to_datetime(df['issue_date'], errors='coerce')
df = df[df.issue_date.dt.year.isin([2018, 2019])]

In [53]:
# Clean up vehicle body type, get rid of null or unknown
# body types, and the least popular ones.
df = df[pd.notnull(df.vehicle_body_type) 
        & (df.vehicle_body_type != 'OLNE')
        & (df.vehicle_body_type != 'LL')
        & (df.vehicle_body_type != 'APUR')
        & (df.vehicle_body_type != 'CW')
        & (df.vehicle_body_type != 'O')
        & (df.vehicle_body_type != 'EP')
        & (df.vehicle_body_type != 'OM')]
df = df.groupby('vehicle_body_type').filter(lambda x: len(x) > 100)
body_types = {
    'SUBN': 'Suburban',
    '4DSD': '4 Door Sedan',
    'VAN': 'Van',
    'DELV': 'Delivery',
    'PICK': 'Pickup Truck',
    '2DSD': '2 Door Sedan',
    'SDN': 'Sedan',
    'REFG': 'Refridgerated',
    'UTIL': 'Utility',
    'TAXI': 'Taxi',
    '4 DR': '4 Door',
    '4D': '4 Door',
    'TRAC': 'Tractor',
    'CONV': 'Convertible',
    'WAGO': 'Wagon',
    'BUS': 'Bus',
    'FLAT': 'Flatbed Truck',
    'P-U': 'Pickup Truck',
    'DUMP': 'Dump Truck',
    'UT': 'Utility',
    'MCY': 'Motorcycle',
    'TRK': 'Truck',
    '4W': '4 Door Wagon',
    'TOW': 'Tow Truck',
    'PKUP': 'Pickup Truck',
    '4S': '4 Door Sedan',
    'SU': 'Sport Utility Vehicle',
    'STAK': 'Stake Truck',
    '2 DR': '2 Door',
    'MP': 'Moped',
    'VN': 'Van',
    'TANK': 'Tank Truck',
    'SW': 'Station Wagon',
    'FOUR': '4 Door',
    'SEDN': 'Sedan',
    'TK': 'Truck',
    '2D': '2 Door',
    'SD': 'Sedan',
    'PK': 'Pickup Truck',
    'SV': 'Sports Van',
    '4H': '4 Door Hatchback',
    'TR/C': 'Truck Crane',
    'CON': 'Convertible',
    'TRUC': 'Truck',
    'LIM': 'Limousine',
    'CP': 'Coupe',
    'SEDA': 'Sedan',
    '4DOO': '4 Door',
    '5D': '5 Door',
    'SWT': 'Small-Wheel Truck',
    'CV': 'Convertible',
    'H/WH': 'Motorhome',
    'CMIX': 'Cement Mixer',
    '4DSE': '4 Door Sedan',
    'TWOD': '2 Door',
    'TRL': 'Trailer',
    'MC': 'Motorcycle',
    'REF': 'Refridgerated',
    'TT': 'Tractor Trailer',
    'TRC': 'Tractor',
    'TRLR': 'Trailer',
    'BOAT': 'Boat Trailer',
    'ES': 'Station Wagon',
    'TRAI': 'Trailer',
    'TR': 'Truck',
    '2S': '2 Door Sedan',
    'HRSE': 'Hearse',
    'TLR': 'Trailer',
    'SPOR': 'Sports Car',
    'SUV': 'SUV',
    'MOT': 'Motorcycle',
    '2H': '2 Door Hatchback',
    'SUBU': 'Suburban',
    'TRT': 'Tractor Trailer',
    'LIMO': 'Limousine',
}
df['vehicle_body_type'] = df['vehicle_body_type'].apply(lambda row: body_types.get(row, row))

In [54]:
# Each street can be encoded with a 5-digit street code
# Each ticket has 3 street codes, because street code 1 
# denotes the "on" street, while street codes 2 & 3
# denote the "cross" steets. For example, Lexington Ave
# between E 42nd and E 45th is a location, where the 
# car was parked on Lexington Ave, between E 42nd and E 45th streets.
# See below link for more detail:
# https://nycplanning.github.io/Geosupport-UPG/chapters/chapterVII/section03/
df = df[(df.street_code1 != 0) & (pd.notnull(df.street_code1))
   & (df.street_code2 != 0) & (pd.notnull(df.street_code2))
   & (df.street_code3 != 0) & (pd.notnull(df.street_code3))]

In [55]:
df = df[(df.vehicle_expiration_date != 0) & (df.vehicle_expiration_date != 88880088)]
df.vehicle_expiration_date = pd.to_datetime(df['vehicle_expiration_date'], format='%Y%m%d',errors='coerce')

In [56]:
# Clean up violation codes. 
# 1-99 are valid codes.
df = df[(df.violation_code > 0) & (df.violation_code < 100)]

In [64]:
# Clean up violation & issuer precincts, 
# get rid of invalid precinct numbers
valid_precincts = [1, 5, 6, 7, 9, 10, 13, 14, 17, 18, 19, 20, 22, 23, 24, 25, 26, 28, 30, 
                   32, 33, 34, 40, 41, 42, 43, 44, 45, 46,47, 48, 49, 50, 52, 60, 61, 
                   62, 63, 66, 67, 68, 69, 70, 71, 72, 73, 75, 76, 77, 78, 79, 81, 83, 
                   84, 88, 90, 94, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 
                   110, 111, 112, 113, 114, 115, 120, 121, 122, 123]
df = df[df.violation_precinct.isin(valid_precincts)]
df = df[df.issuer_precinct.isin(valid_precincts)]

In [92]:
# Clean up violation time by removing
# NaNs and restricting rows to only those that
# match the regex pattern given. This allows
# us to standardize the time format to a 12-hour
# format with "A" or "P" at the end, denoting AM/PM
df = df[(pd.notnull(df.violation_time)) & (df.violation_time.str.contains('(1[012]|0[1-9])[0-5][0-9](A|P)', regex=True, na=False))]

In [100]:
df.head(5)

,registration_state,plate_type,issue_date,violation_code,vehicle_body_type,vehicle_make,issuing_agency,street_code1,street_code2,street_code3,vehicle_expiration_date,violation_precinct,issuer_precinct,violation_time,violation_county,street_name,vehicle_color,vehicle_year,meter?,violation_post_code
1921,NY,Passenger,2018-07-13,40,Sedan,Infiniti,P,20640,18790,19340,2020-01-13,102,102,0511A,Queens,113TH ST,Black,2003,0,NaN
1922,NY,Passenger,2018-07-15,98,Sedan,Hyundai,P,20690,26850,26890,2020-01-21,105,105,0838A,Queens,114 AVE,Gray,2002,0,NaN
1923,NY,Passenger,2018-07-13,46,Sedan,Chevrolet,P,45890,27210,27215,2019-03-03,105,105,0835P,Queens,FRANCIS LEWIS BLVD,Black,2014,0,NaN
1926,NY,Passenger,2018-07-17,46,Suburban,Mercedes-Benz,P,42820,26820,35950,2020-06-07,44,44,1120A,Bronx,JEROME AVE,White,2009,0,NaN
1927,NY,Passenger,2018-07-10,46,Sedan,Toyota,P,26490,36620,52120,2018-11-11,44,44,1236P,Bronx,E 161 ST,Gray,2015,0,NaN
